In [ ]:
####################################################################################################

#    Copyright 2019 Srijan Verma and EMBL-European Bioinformatics Institute

#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at

#        http://www.apache.org/licenses/LICENSE-2.0

#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.

####################################################################################################

In [1]:
import pandas as pd
df_feature_file = pd.read_csv('/feature_selection/seq_align_score_v8/seq_alig_score_ens-ref_v8.csv', index_col=0)

In [9]:
df_feature_file.head(2)

,ENS lincRNA gene ID,Ref lncRNA gene ID,Chr region,No. of transcripts in lincrna,No. of transcripts in lncrna,ENS-Ref Overlap ?,same_opp_overlap,Overlap Length,No. ENS lincrna Exons which overlap RefSeq exons,Max ORF length(ENS),...,Inflated overlap 1kb ens(linc)-ens(protein_coding),is ref LOC ?,Inflated overlap 1kb ref(lnc)-ref(protein_coding),Inflated overlap 0.5kb ens(linc)-ens(linc),Inflated overlap 0.5kb ref(lnc)-ref(lnc),Max exon count,Max exon length,Ens ID for seq alignment score,Ref ID for seq alignment score,Seq Alignment Score
0,ENSG00000236648,gene-LOC107984028,1,1,4,Yes,1,17270,3,90,...,1,1,1,1,1,4,308,ENST00000430540,rna-XR_002958657.1,560.0
1,ENSG00000230704,gene-LOC101928628,1,1,1,Yes,1,1720,2,36,...,0,1,1,1,1,2,677,ENST00000454271,rna-XR_241147.3,677.0


In [2]:
df_ens_ens_overlap = pd.read_csv('/feature_selection/overlap/Ensembl/all_gene_overlap.csv', index_col=0)

In [3]:
df_ens_ens_overlap.head(2)

,lincRNA ID,all gene ID,All Gene biotype,Chr region,No. of transcripts in lincrna,No. of transcripts in all gene,Ensembl-Ensembl overlap ?,same_opp_overlap,Ensembl-linc_Ensembl overlap?
0,ENSG00000230704,ENSG00000000460,protein_coding,1,1,9,Yes,0,No
1,ENSG00000230704,ENSG00000007908,protein_coding,1,1,7,Yes,1,No


In [4]:
from tqdm import tqdm

In [5]:
drop_index = []
for i in tqdm(range(len(df_ens_ens_overlap))):
    if df_ens_ens_overlap['All Gene biotype'][i] == 'protein_coding':
        continue
    else:
        drop_index.append(i)

100%|██████████| 5509/5509 [00:00<00:00, 29545.09it/s]


In [6]:
df_ens_ens_overlap.drop(drop_index, axis=0,inplace=True)
df_ens_ens_overlap.reset_index(drop=True, inplace=True)

In [7]:
len(df_ens_ens_overlap)

1003

In [8]:
set_ens_linc_ids = set(df_ens_ens_overlap['lincRNA ID'].tolist())

In [10]:
protein_ens = []

In [14]:
from tqdm import tqdm

In [15]:
for i in tqdm(range(len(df_feature_file))):
    if df_feature_file['ENS lincRNA gene ID'][i] in set_ens_linc_ids:
        protein_ens.append(0)
    else:
        protein_ens.append(1)

100%|██████████| 4557/4557 [00:00<00:00, 34012.05it/s]


In [17]:
len(protein_ens)

4557

In [18]:
df_ref_ref_overlap = pd.read_csv('/feature_selection/overlap/Refseq/all_gene_overlap.csv', index_col=0)

In [19]:
df_ref_ref_overlap.head(2)

,lncRNA ID,all gene ID,Gene Biotype,Chr region,No. of transcripts in lncrna,No. of transcripts in all gene,RefSeq-RefSeq overlap ?,same_opp_overlap,RefSeq-lnc_RefSeq overlap?
0,gene-MIR1302-2HG,gene-MIR1302-2,miRNA,1,1,2,Yes,1,No
1,gene-LINC01681,gene-ISCUP1,pseudogene,1,1,0,Yes,0,No


In [20]:
drop_index = []
for i in tqdm(range(len(df_ref_ref_overlap))):
    if df_ref_ref_overlap['Gene Biotype'][i] == 'protein_coding':
        continue
    else:
        drop_index.append(i)

100%|██████████| 11278/11278 [00:00<00:00, 36024.24it/s]


In [21]:
df_ref_ref_overlap.drop(drop_index, axis=0,inplace=True)
df_ref_ref_overlap.reset_index(drop=True, inplace=True)

In [22]:
set_ref_lnc_ids = set(df_ref_ref_overlap['lncRNA ID'].tolist())

In [24]:
protein_ref = []

In [25]:
for i in tqdm(range(len(df_feature_file))):
    if df_feature_file['Ref lncRNA gene ID'][i] in set_ref_lnc_ids:
        protein_ref.append(0)
    else:
        protein_ref.append(1) # Means no overlap

100%|██████████| 4557/4557 [00:00<00:00, 36021.44it/s]


In [26]:
df_feature_file['ens(linc)-ens(protein-coding) overlap ?'] = protein_ens
df_feature_file['ref(lnc)-ref(protein-coding) overlap ?'] = protein_ref

In [27]:
df_feature_file.to_csv('/feature_selection/protein_overlap_v9/protein_overlap_v9.csv')